In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from xgboost import XGBClassifier
import time
import pandas as pd
import cv2
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:

train = pd.read_csv('../fashionmnist/fashion-mnist_train.csv')
# test = pd.read_csv('../public_test_dataset/data')
train.isnull().any().sum()
# test.isnull().any().sum()

0

In [4]:
from PIL import Image
import numpy as np
import os
import pandas as pd
import cv2

testset = []
dir_name = '../public_test_dataset/data/'
lists = sorted(os.listdir(dir_name))
for idx, img in enumerate(lists):
    image_gray = cv2.imread(dir_name+img, cv2.IMREAD_GRAYSCALE)
    image = np.array(image_gray.reshape((784,)))
    # image = np.array(Image.open(dir_name+img)).reshape((784,))
    image = image.tolist()
    testset.append(image)
testset = np.array(testset)
test = pd.DataFrame(testset)
test = test/255.0

In [5]:
train.shape

(60000, 785)

In [6]:
test.shape

(10000, 784)

In [7]:
df_train = train.copy()
df_test = test.copy()

In [8]:
X_train= df_train.drop(['label'],axis = 1)
y_train = df_train['label']
X_test = df_test
# y_test = df_test.drop(['label'],axis = 1)
# y_label = df_test['label']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
# y_test /=255.0

In [9]:
# X_train과 X_label을 하나의 데이터 프레임으로 합침
df_train = pd.concat([X_train, y_train], axis=1)

# 데이터 프레임을 섞음
df_train = df_train.sample(frac=1, random_state=42)

# 섞인 데이터 프레임에서 훈련 데이터와 레이블을 다시 분리
X_train = df_train.drop(['label'], axis=1)
y_train = df_train['label']

In [10]:
from skimage.feature import hog
def apply_hog(images):
    result = []
    for image in images:
        hog_features = hog(image, orientations=9, pixels_per_cell=(4, 4),
                        cells_per_block=(2, 2), block_norm='L2')
        result.append(hog_features)
    return result


In [11]:
print(X_train.shape)
print(X_test.shape)

(60000, 784)
(10000, 784)


In [12]:
X_train = np.array(X_train)
X_train = X_train.reshape(-1, 28, 28)
X_test = np.array(X_test)
X_test = X_test.reshape(-1, 28, 28)

In [13]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [14]:
X_train_hog = apply_hog(X_train)
X_test_hog = apply_hog(X_test)
X_train_hog = np.array(X_train_hog)
X_test_hog = np.array(X_test_hog)


In [15]:
print("HOG 특성을 적용한 X_train의 형태: ", np.array(X_train_hog).shape)
print("HOG 특성을 적용한 X_test의 형태: ", np.array(X_test_hog).shape)

HOG 특성을 적용한 X_train의 형태:  (60000, 1296)
HOG 특성을 적용한 X_test의 형태:  (10000, 1296)


In [18]:
from skimage.io import imshow
from skimage import exposure
# 첫 번째 이미지의 HOG 특성 계산 및 시각화
image = X_train[0].reshape((28, 28))
hog_features, hog_image = hog(image, orientations=8, pixels_per_cell=(1, 1),
                              cells_per_block=(2, 2), visualize=True, multichannel=False)

# 원본 이미지와 HOG 특성 이미지를 출력
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

TypeError: hog() got an unexpected keyword argument 'multichannel'

In [16]:
pca = PCA(n_components=250)
pca.fit(X_train_hog)
X_train_pca = pca.transform(X_train_hog)
X_test_pca = pca.transform(X_test_hog)

X_train_PCA1 = pd.DataFrame(X_train_pca)
X_test_PCA1 = pd.DataFrame(X_test_pca)

In [17]:
optimal_c = 8
svc = SVC(gamma='scale',kernel='rbf',C=optimal_c )
svc.fit(X_train_PCA1,y_train)

SVC(C=8)

In [18]:
svc_train = svc.predict(X_train_PCA1)
svc_pred = svc.predict(X_test_PCA1)

In [ ]:
# pred = model_load.predict(test)
f= open("../testResults/testResult_public_hog_n400_c8.txt","w+")
for idx, y in enumerate(svc_pred):
    num_str = str(idx).zfill(5)
    f.write(num_str + " " + str(int(y)) + "\n")
f.close()

In [ ]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

# testResult_path = sys.argv[1]
# label_path = sys.argv[2]

testResult_path = '../testResults/testResult_public_hog_n400_c8.txt'
label_path = '../mAP/label.txt'

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    print("TP :", TP)
    print("FN :", FN)
    # print(temp_precision)
    # print(temp_recall)
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.2f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.2f} |\n".format("mAP", mAP)
test_mAP = mAP
print(table)


In [ ]:
from sklearn.metrics import auc
from collections import Counter
def calculate_mAP(preds,label):
    ## mAP calculation
    AP = []
    num_class = 10
    predict_label_count_dict = Counter(preds)
    predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

    # For each class
    for c, freq in predict_label_count_dict.items() :
        TP = 0
        FN = 0

        temp_precision = []
        temp_recall = []

        for i in range(len(preds)):
            # Calculate TP and FN
            if label[i] == c and preds[i] == c :
                TP += 1
            elif label[i] != c and preds[i] == c :
                FN += 1

            # Calculate precision and recall
            if TP+FN != 0:
                temp_precision.append(TP/(TP+FN))
                temp_recall.append(TP/freq)

        # Save the AP value of each class to AP array
        AP.append(auc(temp_recall, temp_precision))

    # Calculate mAP
    mAP = sum(AP) / num_class

    return mAP

In [82]:
# train_acc = metrics.accuracy_score(y_train,svc_train)
map_train = calculate_mAP(svc_train,y_train)
print("Train Map score: {}".format(map_train))
print("Test Map score: {}".format(test_mAP))

Train Map score: 0.9985790382150592
Test Map score: 0.8514137160302695
